In [1]:
import os

from huggingface_hub import login
from omegaconf import OmegaConf

from lema.builders import (
    build_dataset,
    build_model,
    build_peft_model,
    build_tokenizer,
    build_trainer,
)
from lema.core.types import TrainingConfig
from lema.utils.saver import save_model

%load_ext autoreload
%autoreload 2

In [2]:
access_token = os.environ.get("HF_TOKEN")
login(token=access_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/optas/.cache/huggingface/token
Login successful


In [3]:
config_filename = "../configs/lema/zephyr.7b.sft.yaml"
base_config = OmegaConf.structured(TrainingConfig)
file_config = TrainingConfig.from_yaml(config_filename)
config = OmegaConf.merge(base_config, file_config)
config: TrainingConfig = OmegaConf.to_object(config)
print(config.training)
print(config.peft)

TrainingParams(optimizer='adamw_torch', use_peft=True, trainer_type=<TrainerType.TRL_SFT: 'trl_sft'>, enable_gradient_checkpointing=True, output_dir='output/zephyr.7b.sft', per_device_train_batch_size=1, per_device_eval_batch_size=8, gradient_accumulation_steps=1, max_steps=2, num_train_epochs=1, run_name='default', log_level='info', dep_log_level='warning', enable_wandb=False, enable_tensorboard=True, logging_strategy='steps', logging_dir='output/runs', logging_steps=50, learning_rate=2e-05, lr_scheduler_type='cosine', warmup_ratio=0.1, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, gradient_checkpointing_kwargs={'use_reentrant': False}, include_performance_metrics=None)
PeftParams(lora_r=16, lora_alpha=16, lora_dropout=0.05, lora_target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'], lora_modules_to_save=None, lora_bias='none', lora_task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, q_lora=False, q_lora_bits=4)


In [4]:
# TODO-finalize in config file
config.peft.q_lora = False
config.training.per_device_train_batch_size = 1
config.training.max_steps = 2

In [5]:
tokenizer = build_tokenizer(config.model)
tokenizer

/Users/optas/miniconda3/envs/lema/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
[2024-06-06 00:11:40,071][lema][WARNING][models.py:120] <pad> token not found: setting <pad> with <eos>.


LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-v0.1', vocab_size=32000, model_max_length=2048, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [6]:
# if tokenizer.model_max_length > 100_000: # shall this condition be checked for diff.
#  Zephyr models? Now is not.

print("tokenizer.model_max_length", tokenizer.model_max_length)
print("tokenizer pad_token/eos_token", tokenizer.pad_token, tokenizer.eos_token)
print("tokenizer.padding_side", tokenizer.padding_side)
print("tokenizer.chat_template", tokenizer.chat_template)

tokenizer.model_max_length 2048
tokenizer pad_token/eos_token </s> </s>
tokenizer.padding_side left
tokenizer.chat_template {% if messages[0]['role'] == 'system' %}{% set offset = 1 %}{% else %}{% set offset = 0 %}{% endif %}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == offset) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{{ '<|' + message['role'] + '|>\n' + message['content'] | trim + eos_token + '\n' }}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>\n' }}{% endif %}


In [7]:
# Load data & preprocessing
dataset = build_dataset(data_params=config.data, tokenizer=tokenizer)


if True:
    import numpy as np  # hack to subsample

    print(len(dataset))
    np.random.seed(1234)
    ridx = np.random.choice(len(dataset), 1024, replace=False)
    dataset = dataset.select(ridx)
    print(len(dataset))

dataset

TypeError: build_dataset() missing 1 required positional argument: 'data_params'

In [ ]:
# TODO - update our code base if we use optimum
# Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file. # noqa
# WARNING:auto_gptq.nn_modules.qlinear.qlinear_cuda:CUDA extension not installed. # TODO update in main repo # noqa

In [ ]:
model = build_model(config)

In [ ]:
tokenizer.all_special_tokens
tokenizer.encode("|system|")

# TODO Consider adding special tokens like '<|assistant|>', '<|system|>'
# via tokenizer.additional_special_tokens -- need to check Mistral

# from alignment team:
# tokenizer.encode("<|system|>")  # We already wrap <bos> and <eos>
# # in the chat template

# Future TODO.
# # For ChatML we need to add special tokens and resize the embedding layer
# if "<|im_start|>" in tokenizer.chat_template and "gemma-tokenizer-chatml" not in tokenizer.name_or_path: # noqa
#     model = AutoModelForCausalLM.from_pretrained(model_args.model_name_or_path, **model_kwargs) # noqa
#     model, tokenizer = setup_chat_format(model, tokenizer)
#     model_kwargs = None

In [ ]:
if config.training.use_peft:
    model = build_peft_model(
        model, config.training.enable_gradient_checkpointing, config.peft
    )

if config.training.enable_gradient_checkpointing:
    model.enable_input_require_grads()

In [ ]:
# TODO update if need be for accelerator
trainer_cls = build_trainer(config.training.trainer_type)

In [ ]:
trainer = trainer_cls(
    model=model,
    tokenizer=tokenizer,
    args=config.training.to_hf(),
    train_dataset=dataset,
    **config.data.trainer_kwargs,
)

In [ ]:
dataset[0]["text"]

In [ ]:
trainer.train()

In [ ]:
# Save final checkpoint & training state
trainer.save_state()

save_model(
    config=config,
    trainer=trainer,
)